# Exercise versus

In [1]:
# To list files in a directory
import os
# Move up in directories
print(os.getcwd()) # for debugging
os.chdir("..")
print(os.getcwd()) # for debugging

# To work with .json files
import json

# To work with .csv files
import pandas as pd

# To parse dates
from datetime import datetime, timedelta

# To time execution
import time # for debugging

C:\Users\leyjf\Dropbox\github\FitbitStatsProject\CleanedData
C:\Users\leyjf\Dropbox\github\FitbitStatsProject


In [2]:
# Load and parse exercise file
exercise_list = open("CleanedData\\exercise_new_grouped_by_date.json")
#type(exercise_list) # for debugging
parsed_exercise = json.load(exercise_list)
#parsed_exercise # for debugging
exercise_list.close()

## Exercise versus resting heartrate

In [3]:
# Load sleep scores
sleep_scores = pd.read_csv("RawData\\ConsolidatedData\\sleep_score.csv", index_col = [0])
#sleep_scores # for debugging

# Match date format
for date in sleep_scores["timestamp"]:
    #print(date) # for debugging
    parsed_date = datetime.strptime(date, "%Y-%m-%dT%H:%M:%SZ").strftime('%m/%d/%y')
    #print(parsed_date) # for debugging
    sleep_scores["timestamp"] = sleep_scores["timestamp"].replace(date, parsed_date)
#sleep_scores # for debugging

# Replace multiple rows with the same date with the averages
averaged_sleep_scores = sleep_scores.groupby(["timestamp"]).mean()
#averaged_sleep_scores # for debugging

# New .json with date, total exercise, resting heartrate
averaged_sleep_scores_string = averaged_sleep_scores.to_json()
averaged_sleep_scores_json = json.loads(averaged_sleep_scores_string)
#print(averaged_sleep_scores_json) # for debugging
#print(type(averaged_sleep_scores_json)) # for debugging
exercise_v_resting_hr = []
for entry in parsed_exercise:
    if entry["date"] not in ['03/15/22','04/05/22','07/28/22','08/23/22','09/12/22','09/21/22',
                            '10/31/22','11/14/22','02/07/23']:
        month_year = datetime.strptime(entry["date"], "%m/%d/%y").strftime('%B %Y')
        new_entry = {
            "date" : entry["date"],
            "resting_hr" : averaged_sleep_scores_json["resting_heart_rate"][entry["date"]],
            "total_exercise" : entry["totalMinutes"],
            "month": month_year
        }
        #print(new_entry) # for debugging
        exercise_v_resting_hr.append(new_entry)
#resting_hr_v_exercise # for debugging        

# Save to a file
exercise_v_resting_hr_string = json.dumps(exercise_v_resting_hr)
exercise_v_resting_hr_json = json.loads(exercise_v_resting_hr_string)
save_file = open("CleanedData\\exercise_v_resting_hr.json", "w")  
json.dump(exercise_v_resting_hr_json, save_file, indent = 4)  
save_file.close()

## Exercise versus sleep score

In [4]:
#averaged_sleep_scores_json # for debugging
#parsed_exercise # for debugging
exercise_v_sleep_score = []
for entry in parsed_exercise:
    if entry["date"] not in ['03/15/22','04/05/22','07/28/22','08/23/22','09/12/22','09/21/22',
                            '10/31/22','11/14/22','02/07/23']:
        month_year = datetime.strptime(entry["date"], "%m/%d/%y").strftime('%B %Y')
        new_entry = {
            "date" : entry["date"],
            "total_exercise" : entry["totalMinutes"],
            "sleep_score" : averaged_sleep_scores_json["overall_score"][entry["date"]],
            "composition" : averaged_sleep_scores_json["composition_score"][entry["date"]],
            "revitalization" : averaged_sleep_scores_json["revitalization_score"][entry["date"]],
            "duration" : averaged_sleep_scores_json["duration_score"][entry["date"]],
            "month": month_year
        }
    #print(new_entry) # for debugging
    exercise_v_sleep_score.append(new_entry)
#resting_hr_v_exercise # for debugging        

# Save to a file
exercise_v_sleep_score_string = json.dumps(exercise_v_sleep_score)
exercise_v_sleep_score_json = json.loads(exercise_v_sleep_score_string)
save_file = open("CleanedData\\exercise_v_sleep_score.json", "w")  
json.dump(exercise_v_sleep_score_json, save_file, indent = 4)  
save_file.close()

## Exercise versus sleep stages

In [11]:
# Load and parse sleep scores
sleep_stages_list = open("RawData\\ConsolidatedData\\sleep_stages.json")
parsed_stages = json.load(sleep_stages_list)
sleep_stages_list.close()
#parsed_stages # for debugging

# Match date format
for entry in parsed_stages:
    date = entry["startTime"]
    parsed_date = datetime.strptime(date, "%Y-%m-%dT%H:%M:%S.%f").strftime('%m/%d/%y')
    #print(parsed_date) # for debugging
    entry["startTime"] = entry["startTime"].replace(date, parsed_date)
#parsed_stages # for debugging

exercise_v_sleep_stages = []
for entry in parsed_exercise:
    #print(entry) # for debugging
    indiv_entries = []
    for log in parsed_stages:
        if log["startTime"]==entry["date"]:
            #print(log) # for debugging
            if "deep" in log["levels"]["summary"]:
                instance = {
                    "date": log["startTime"],
                    "deep_sleep" : log["levels"]["summary"]["deep"]["minutes"],
                    "light_sleep" : log["levels"]["summary"]["light"]["minutes"],
                    "rem_sleep" : log["levels"]["summary"]["rem"]["minutes"]
                }
                #print(indiv_entries) # for debugging
                indiv_entries.append(instance)    
    deep_sleep = 0
    light_sleep = 0
    rem_sleep = 0
    #print(indiv_entries) # for debugging
    for indiv_entry in indiv_entries:
        deep_sleep = deep_sleep +indiv_entry["deep_sleep"]
        light_sleep = light_sleep +indiv_entry["light_sleep"]
        rem_sleep = rem_sleep +indiv_entry["rem_sleep"]
        month_year = datetime.strptime(entry["date"], "%m/%d/%y").strftime('%B %Y')
    new_entry = {
        "date": entry["date"],
        "total_exercise": entry["totalMinutes"],
        "deep_sleep": deep_sleep,
        "light_sleep": light_sleep,
        "rem_sleep": rem_sleep,
        "total_sleep": (deep_sleep+light_sleep+rem_sleep)/60,
        "month": month_year
    }
    #print(new_entry) # for debugging
    if new_entry["total_sleep"] != 0:
        exercise_v_sleep_stages.append(new_entry)
exercise_v_sleep_stages # for debugging        

# Save to a file
exercise_v_sleep_stages_string = json.dumps(exercise_v_sleep_stages)
exercise_v_sleep_stages_json = json.loads(exercise_v_sleep_stages_string)
save_file = open("CleanedData\\exercise_v_sleep_stages.json", "w")  
json.dump(exercise_v_sleep_stages_json, save_file, indent = 4)  
save_file.close()

{'date': '03/15/22', 'total_exercise': 83, 'deep_sleep': 120, 'light_sleep': 228, 'rem_sleep': 87, 'total_sleep': 7.25, 'month': 'March 2022'}
{'date': '03/16/22', 'total_exercise': 23, 'deep_sleep': 63, 'light_sleep': 331, 'rem_sleep': 79, 'total_sleep': 7.883333333333334, 'month': 'March 2022'}
{'date': '03/17/22', 'total_exercise': 72, 'deep_sleep': 51, 'light_sleep': 309, 'rem_sleep': 35, 'total_sleep': 6.583333333333333, 'month': 'March 2022'}
{'date': '03/20/22', 'total_exercise': 10, 'deep_sleep': 155, 'light_sleep': 530, 'rem_sleep': 97, 'total_sleep': 13.033333333333333, 'month': 'March 2022'}
{'date': '03/21/22', 'total_exercise': 24, 'deep_sleep': 115, 'light_sleep': 176, 'rem_sleep': 104, 'total_sleep': 6.583333333333333, 'month': 'March 2022'}
{'date': '03/22/22', 'total_exercise': 47, 'deep_sleep': 82, 'light_sleep': 195, 'rem_sleep': 99, 'total_sleep': 6.266666666666667, 'month': 'March 2022'}
{'date': '03/23/22', 'total_exercise': 51, 'deep_sleep': 0, 'light_sleep': 0, 